In [1]:
import numpy as np
import pandas as pd
import random as rd
######Stanford Sentiment Treebank######################
sstb = pd.read_csv("datasetSentences.txt", sep = "\t", usecols=[1])
datasetsplit = pd.read_csv("datasetSplit.txt", sep = ",", usecols=[1])

sstb = pd.concat([sstb,datasetsplit], axis=1)

sstb_train = sstb[sstb["splitset_label"] == 1]
sstb_test = sstb[sstb["splitset_label"] == 2]
sstb_validation = sstb[sstb["splitset_label"] == 3]
##########Stanford Twitter Sentiment Corpus#############

skip = sorted(rd.sample(range(1600000), 1600000-80000))
sts_train = pd.read_csv("training.csv", encoding = "ISO-8859-1", engine='python', sep=",", header=None, skiprows = skip)
sts_test = pd.read_csv("testdata.csv", sep=",", header = None)
skip = sorted(rd.sample(range(1600000), 1600000-16000))
sts_validation = pd.read_csv("training.csv", encoding = "ISO-8859-1", engine="python", sep=",", header=None, skiprows = skip)



In [2]:
sts_validation

,0,1,2,3,4,5
0,0,1467837384,Mon Apr 06 22:26:42 PDT 2009,NO_QUERY,kenandise,Behind on my classes for work
1,0,1467844140,Mon Apr 06 22:28:32 PDT 2009,NO_QUERY,Twokids1,@rumblepurr lol.. wish they understood dayligh...
2,0,1467889231,Mon Apr 06 22:40:24 PDT 2009,NO_QUERY,luannem,@kariajay All this time you didn't notice I w...
3,0,1467900244,Mon Apr 06 22:43:26 PDT 2009,NO_QUERY,Mowgli3,"http://twitpic.com/2y2yi - I love you, Buck."
4,0,1467916510,Mon Apr 06 22:47:56 PDT 2009,NO_QUERY,Daniiej,omg i've an economics test. and i dont know al...
...,...,...,...,...,...,...
15995,4,2193346429,Tue Jun 16 08:20:01 PDT 2009,NO_QUERY,ChanelBlueSatin,@Neevy89 lol well people say you can get the s...
15996,4,2193474371,Tue Jun 16 08:30:27 PDT 2009,NO_QUERY,H_Anderson3,@sundayrain haha. Thanks.. Yeah I got that tat...
15997,4,2193476073,Tue Jun 16 08:30:35 PDT 2009,NO_QUERY,xLukesterx,@NicciLuvsMollie no problem ;) and awww I'm s...
15998,4,2193576427,Tue Jun 16 08:38:45 PDT 2009,NO_QUERY,wonder_nat,@AndrewDearling *yawns*


In [3]:
sts_train.head()

,0,1,2,3,4,5
0,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
1,0,1467816149,Mon Apr 06 22:21:11 PDT 2009,NO_QUERY,Pbearfox,@julieebaby awe i love you too!!!! 1 am here ...
2,0,1467820906,Mon Apr 06 22:22:24 PDT 2009,NO_QUERY,voyage2k,"@localtweeps Wow, tons of replies from you, ma..."
3,0,1467825084,Mon Apr 06 22:23:30 PDT 2009,NO_QUERY,PresidentSnow,"@Lt_Algonquin agreed, I saw the failwhale alll..."
4,0,1467833799,Mon Apr 06 22:25:46 PDT 2009,NO_QUERY,kaelaaa,i think my arms are sore from tennis


In [4]:
sts_train.shape #80,000 rows and 6 columns 

(80000, 6)

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer 

In [6]:
sts_train.isnull().mean().sort_values(ascending=False)

5    0.0
4    0.0
3    0.0
2    0.0
1    0.0
0    0.0
dtype: float64

In [7]:
sts_train[5].value_counts(dropna=False)

isPlayer Has Died! Sorry                                                                                                               10
Goodnight                                                                                                                               7
headache                                                                                                                                7
Headache                                                                                                                                6
I am so so so so so bored of studying                                                                                                   5
                                                                                                                                       ..
800 words, and screen shots in under 30min, that has to be a new record for me. Moving Image assignment done, just to finish radio      1
@ChrisHardie  you might want to lo

In [8]:
sts_train[5].head()

0                                          Need a hug 
1    @julieebaby awe i love you too!!!! 1 am here  ...
2    @localtweeps Wow, tons of replies from you, ma...
3    @Lt_Algonquin agreed, I saw the failwhale alll...
4                i think my arms are sore from tennis 
Name: 5, dtype: object

In [9]:
# sts_train = sts_train.drop([0,1,3], axis=1)

In [10]:
sts_train

,0,1,2,3,4,5
0,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
1,0,1467816149,Mon Apr 06 22:21:11 PDT 2009,NO_QUERY,Pbearfox,@julieebaby awe i love you too!!!! 1 am here ...
2,0,1467820906,Mon Apr 06 22:22:24 PDT 2009,NO_QUERY,voyage2k,"@localtweeps Wow, tons of replies from you, ma..."
3,0,1467825084,Mon Apr 06 22:23:30 PDT 2009,NO_QUERY,PresidentSnow,"@Lt_Algonquin agreed, I saw the failwhale alll..."
4,0,1467833799,Mon Apr 06 22:25:46 PDT 2009,NO_QUERY,kaelaaa,i think my arms are sore from tennis
...,...,...,...,...,...,...
79995,4,2193576360,Tue Jun 16 08:38:44 PDT 2009,NO_QUERY,MandyStocker,dropping molly off getting ice cream with Aaro...
79996,4,2193576815,Tue Jun 16 08:38:47 PDT 2009,NO_QUERY,sarasmile13,@gone2dmb agreed! Soulful eyes.
79997,4,2193577517,Tue Jun 16 08:38:51 PDT 2009,NO_QUERY,stephyway,oooo haha just waking up and ready to eat a de...
79998,4,2193577671,Tue Jun 16 08:38:51 PDT 2009,NO_QUERY,craiglparker,@siovene lol I don't blame you it's not the sa...


In [11]:
# sts_train_copy.dtypes

In [12]:
#polarity: 0 = negative, 2 = neutral, 4 = positive
sts_train.rename(columns={0:'polarity', 1: 'tweet_id', 2:'date', 4:'username', 5:'tweet_text'}, inplace=True)
sts_validation.rename(columns={0:'polarity', 1: 'tweet_id', 2:'date', 4:'username', 5:'tweet_text'}, inplace=True)
sts_test.rename(columns={0:'polarity', 1: 'tweet_id', 2:'date', 4:'username', 5:'tweet_text'}, inplace=True)

In [13]:
sts_train = sts_train.drop([3], axis=1)


In [14]:
sts_train

,polarity,tweet_id,date,username,tweet_text
0,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,mybirch,Need a hug
1,0,1467816149,Mon Apr 06 22:21:11 PDT 2009,Pbearfox,@julieebaby awe i love you too!!!! 1 am here ...
2,0,1467820906,Mon Apr 06 22:22:24 PDT 2009,voyage2k,"@localtweeps Wow, tons of replies from you, ma..."
3,0,1467825084,Mon Apr 06 22:23:30 PDT 2009,PresidentSnow,"@Lt_Algonquin agreed, I saw the failwhale alll..."
4,0,1467833799,Mon Apr 06 22:25:46 PDT 2009,kaelaaa,i think my arms are sore from tennis
...,...,...,...,...,...
79995,4,2193576360,Tue Jun 16 08:38:44 PDT 2009,MandyStocker,dropping molly off getting ice cream with Aaro...
79996,4,2193576815,Tue Jun 16 08:38:47 PDT 2009,sarasmile13,@gone2dmb agreed! Soulful eyes.
79997,4,2193577517,Tue Jun 16 08:38:51 PDT 2009,stephyway,oooo haha just waking up and ready to eat a de...
79998,4,2193577671,Tue Jun 16 08:38:51 PDT 2009,craiglparker,@siovene lol I don't blame you it's not the sa...


In [16]:
x = sts_train.tweet_text
y = sts_train.polarity

In [18]:
from sklearn.model_selection import train_test_split
# SEED = 2000
# x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(x, y, test_size=.02, random_state=SEED)
# x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, test_size=.5, random_state=SEED)

In [19]:
# x_train


19596    wonders why the change of status approval noti...
30172    New computer get here NOW. I don't want to wai...
49792    Belanja Kaos itu gampang! www.kaoskita.com T-s...
27870    just saw an ad for rush *tear* I cried so much...
58902    Congratulations @garyvee, Lizzie and family &a...
                               ...                        
79483                 @LVMAKEUP we had a great weekend... 
34358       @this_is_tors just a cheeky 17. My knee hurts 
69581    just finished miXINgTpes   ha!! time to sleep ...
62670          Watching my favorite people play softball. 
37704    Ughhh. Twitter is not playing nice again this ...
Name: tweet_text, Length: 78400, dtype: object

0        0
1        0
2        0
3        0
4        0
        ..
79995    4
79996    4
79997    4
79998    4
79999    4
Name: polarity, Length: 80000, dtype: int64

In [20]:
# y_train

19596    0
30172    0
49792    4
27870    0
58902    4
        ..
79483    4
34358    0
69581    4
62670    4
37704    0
Name: polarity, Length: 78400, dtype: int64

In [21]:


# print("Train set has total {0} entries with {1:.2f}% negative, {1:.2f}% positive, {1:.2f}% neutral".format(len(x),
#                                                                     (len(x[y == 0]) / (len(x)*1.))*100,
#                                                                     (len(x[y == 4]) / (len(x)*1.))*100,
#                                                                     (len(x[y == 2]) / (len(x)*1.))*100))


In [22]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import gensim
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import TaggedDocument
import multiprocessing
from sklearn import utils

In [23]:
def labelize_tweets_ug(tweets,label):
    result = []
    prefix = label
    for i, t in zip(tweets.index, tweets):
        result.append(TaggedDocument(t.split(), [prefix + '_%s' % i]))
    return result

In [24]:
x_validation = sts_validation.tweet_text
x_validation

0                           Behind on my classes for work 
1        @rumblepurr lol.. wish they understood dayligh...
2        @kariajay  All this time you didn't notice I w...
3            http://twitpic.com/2y2yi - I love you, Buck. 
4        omg i've an economics test. and i dont know al...
                               ...                        
15995    @Neevy89 lol well people say you can get the s...
15996    @sundayrain haha. Thanks.. Yeah I got that tat...
15997    @NicciLuvsMollie no problem ;) and awww  I'm s...
15998                             @AndrewDearling *yawns* 
15999    happy #charitytuesday @theNSPCC @SparksCharity...
Name: tweet_text, Length: 16000, dtype: object

In [35]:
y_validation = sts_validation.polarity
y_validation

0        0
1        0
2        0
3        0
4        0
        ..
15995    4
15996    4
15997    4
15998    4
15999    4
Name: polarity, Length: 16000, dtype: int64

In [25]:
x_test = sts_test.tweet_text
x_test

0      @stellargirl I loooooooovvvvvveee my Kindle2. ...
1      Reading my kindle2...  Love it... Lee childs i...
2      Ok, first assesment of the #kindle2 ...it fuck...
3      @kenburbary You'll love your Kindle2. I've had...
4      @mikefish  Fair enough. But i have the Kindle2...
                             ...                        
493    Ask Programming: LaTeX or InDesign?: submitted...
494    On that note, I hate Word. I hate Pages. I hat...
495    Ahhh... back in a *real* text editing environm...
496    Trouble in Iran, I see. Hmm. Iran. Iran so far...
497    Reading the tweets coming out of Iran... The w...
Name: tweet_text, Length: 498, dtype: object

In [36]:
y_test = sts_test.polarity
y_test

0      4
1      4
2      4
3      4
4      4
      ..
493    2
494    0
495    4
496    0
497    0
Name: polarity, Length: 498, dtype: int64

In [26]:
all_x = pd.concat([x, x_validation, x_test])
all_x_w2v = labelize_tweets_ug(all_x, 'all')

In [27]:
cores = multiprocessing.cpu_count()
model_ug_cbow = Word2Vec(sg=0, size=100, negative=5, window=2, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_cbow.build_vocab([x.words for x in tqdm(all_x_w2v)])

100%|██████████| 96498/96498 [00:00<00:00, 2154120.17it/s]


In [28]:
%%time
for epoch in range(30):
    model_ug_cbow.train(utils.shuffle([x.words for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_cbow.alpha -= 0.002
    model_ug_cbow.min_alpha = model_ug_cbow.alpha

100%|██████████| 96498/96498 [00:00<00:00, 2483734.65it/s]


CPU times: user 1min 16s, sys: 717 ms, total: 1min 17s
Wall time: 31.5 s


In [29]:
model_ug_sg = Word2Vec(sg=1, size=100, negative=5, window=2, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_sg.build_vocab([x.words for x in tqdm(all_x_w2v)])

100%|██████████| 96498/96498 [00:00<00:00, 2252357.84it/s]


In [30]:
%%time
for epoch in range(30):
    model_ug_sg.train(utils.shuffle([x.words for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_sg.alpha -= 0.002
    model_ug_sg.min_alpha = model_ug_sg.alpha

100%|██████████| 96498/96498 [00:00<00:00, 2337360.09it/s]


CPU times: user 2min 26s, sys: 831 ms, total: 2min 27s
Wall time: 44.8 s


In [31]:
model_ug_cbow.save('w2v_model_ug_cbow.word2vec')
model_ug_sg.save('w2v_model_ug_sg.word2vec')

In [37]:
#CNN preparation

In [38]:
from gensim.models import KeyedVectors
model_ug_cbow = KeyedVectors.load('w2v_model_ug_cbow.word2vec')
model_ug_sg = KeyedVectors.load('w2v_model_ug_sg.word2vec')

In [39]:
len(model_ug_cbow.wv.vocab.keys())

42019

In [40]:
embeddings_index = {}
for w in model_ug_cbow.wv.vocab.keys():
    embeddings_index[w] = np.append(model_ug_cbow.wv[w],model_ug_sg.wv[w])
print('Found %s word vectors.' % len(embeddings_index))

Found 42019 word vectors.


In [41]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(x)
sequences = tokenizer.texts_to_sequences(x)

Using TensorFlow backend.
/Applications/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Applications/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Applications/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Applications/anaconda3/lib/python3.6/site-packages/t

In [42]:
len(tokenizer.word_index)


84337

In [43]:
for words in x[:5]:
    print(words)

Need a hug 
@julieebaby awe i love you too!!!! 1 am here  i miss you
@localtweeps Wow, tons of replies from you, may have to unfollow so I can see my friends' tweets, you're scrolling the feed a lot. 
@Lt_Algonquin agreed, I saw the failwhale allllll day today. 
i think my arms are sore from tennis 


In [44]:
#Same data as sequential data 
sequences[:5] #Each word is represented as a number 


[[90, 4, 1071],
 [23976, 1723, 1, 44, 7, 45, 195, 63, 91, 1, 87, 7],
 [23977,
  258,
  2057,
  12,
  2341,
  54,
  7,
  331,
  16,
  2,
  5105,
  15,
  1,
  66,
  69,
  5,
  9974,
  445,
  157,
  15364,
  3,
  1621,
  4,
  299],
 [156, 23978, 2058, 1, 309, 3, 15365, 15366, 30, 40],
 [1, 78, 5, 1979, 36, 619, 54, 1946]]

In [45]:
length = []
x2 = x
for words in x:
    length.append(len(words.split()))

In [46]:
max(length)

44

In [47]:
x_train_seq = pad_sequences(sequences, maxlen=50)
print('Shape of data tensor:', x_train_seq.shape)

Shape of data tensor: (80000, 50)


In [48]:
sequences_val = tokenizer.texts_to_sequences(x_validation)
x_val_seq = pad_sequences(sequences_val, maxlen=50)


In [49]:
x_train_seq[:5]


array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,    90,     4,  1071],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0, 23976,  1723,     1,    44,     7,    45,   195,
           63,    91,     1,    87,     7],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,    

In [56]:
num_words = 100000
embedding_matrix = np.zeros((num_words, 200))
for word, i in tokenizer.word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [57]:
np.array_equal(embedding_matrix[1071] ,embeddings_index.get('hug'))

True

In [58]:
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

In [59]:
structure_test = Sequential()
e = Embedding(100000, 200, input_length=50)
structure_test.add(e)
structure_test.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
structure_test.add(GlobalMaxPooling1D())

structure_test.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 50, 200)           20000000  
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 49, 100)           40100     
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 100)               0         
Total params: 20,040,100
Trainable params: 20,040,100
Non-trainable params: 0
_________________________________________________________________


In [64]:
len(y)

80000

In [65]:
model_cnn_02 = Sequential()
e = Embedding(100000, 200, input_length=50)
model_cnn_02.add(e)
model_cnn_02.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_02.add(GlobalMaxPooling1D())
model_cnn_02.add(Dense(256, activation='relu'))
model_cnn_02.add(Dense(1, activation='sigmoid'))
model_cnn_02.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_cnn_02.fit(x_train_seq, y, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=32, verbose=2)

/Applications/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/indexed_slices.py:414: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 80000 samples, validate on 16000 samples
Epoch 1/5


KeyboardInterrupt: 